In [12]:
# Reg fetch new batch of features and compute predictions and save to feature store
# 

In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [15]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

2025-03-03 19:37:31,385 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:37:31,388 INFO: Initializing external client
2025-03-03 19:37:31,389 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:37:32,221 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652
Fetching data from 2025-02-03 00:37:31.385739+00:00 to 2025-03-03 23:37:31.385739+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.05s) 


In [16]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 19:37:46,534 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:37:46,537 INFO: Initializing external client
2025-03-03 19:37:46,537 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:37:47,119 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652

In [17]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,1,1,0,0,1,1,2,1,2,0,...,0,1,0,0,0,0,0,0,196,2025-02-18 03:00:00
1,32,108,136,24,123,145,12,229,57,130,...,131,8,16,57,25,106,41,132,263,2025-02-14 08:00:00
2,2,0,1,3,0,0,0,24,0,0,...,1,0,0,2,0,1,3,0,255,2025-02-27 19:00:00
3,0,0,0,1,0,1,0,0,0,1,...,0,0,0,1,1,0,0,0,210,2025-02-10 09:00:00
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,53,2025-02-11 20:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,1,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,1,122,2025-02-21 08:00:00
247,47,173,243,3,177,148,423,199,173,156,...,5,198,232,29,149,75,206,242,142,2025-02-12 11:00:00
248,67,70,69,81,75,122,165,22,93,98,...,55,37,124,60,124,121,74,124,231,2025-02-16 06:00:00
249,0,0,0,0,0,1,0,0,0,1,...,0,3,0,2,0,0,2,2,205,2025-03-01 12:00:00


In [18]:
model

Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x13ae03e20>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [19]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [20]:
predictions["pickup_hour"] = current_date.ceil('h')
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,196,1.0,2025-03-04 01:00:00+00:00
1,263,88.0,2025-03-04 01:00:00+00:00
2,255,1.0,2025-03-04 01:00:00+00:00
3,210,0.0,2025-03-04 01:00:00+00:00
4,53,0.0,2025-03-04 01:00:00+00:00
...,...,...,...
246,122,0.0,2025-03-04 01:00:00+00:00
247,142,178.0,2025-03-04 01:00:00+00:00
248,231,100.0,2025-03-04 01:00:00+00:00
249,205,0.0,2025-03-04 01:00:00+00:00


In [21]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["pickup_location_id", "pickup_hour"],
    event_time="pickup_hour",
)

2025-03-03 19:37:59,239 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 19:37:59,241 INFO: Initializing external client
2025-03-03 19:37:59,242 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 19:38:00,071 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215652


In [22]:
feature_group.insert(predictions, write_options={"wait_for_job": False})


ploading Dataframe: 100.00% |█| Rows 251/251 | Elapsed Time: 00:01 | Remaining 

Launching job: taxi_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1215652/jobs/named/taxi_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('taxi_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)